# Проверка работы модели на новых данных

In [1]:
# Импорт неоходимых библиотек
import pandas as pd
import re

# Векторизация текстовых данных TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Нормализация по длине документа
from sklearn.preprocessing import normalize
# Преобразование в разреженную матрицу
from scipy.sparse import csr_matrix

# Метрики
from sklearn.metrics import (accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve)

# Понижение размерности
from sklearn.manifold import TSNE

# Лингивистические модули
from razdel import tokenize # сегментация русскоязычного текста на токены и предложения
import pymorphy2  # Морфологический анализатор
import nltk # лингивистический модуль nltk 
from nltk.corpus import stopwords # импортируем стоп-слова

In [2]:
# Подготовка функций

# Функция разметки коментариев с использованим поля stars (от 1-3 негивный отзыв, от 4-5 позитивный отзыв)
def get_label_target(num_stars: int) -> int:
    if num_stars in (1, 2, 3):
        return 0  # отзыв негативный
    else:
        return 1  # отзыв позитивный


# Функция очистки отзывов от лишних пробелов, символов и тд. и тп
def clean_text(text):
    '''
    очистка текста
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))
    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text


# Функция токенизации, лемматизация, удаления стоп-слов
cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text: str) -> str:
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова
    на выходе лист отлемматизированых токенов
    '''
    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopwords_ru] # [6]
    
    return words_lem_without_stopwords

# Создадим список стоп-слов из модуля NLTK
nltk.download('stopwords')
stopwords_ru = stopwords.words('russian')

# Уберем частицу не из словаря стоп-слов
stopwords_ru.remove('не')

# Дополним стоп-слова из внешнего локального словаря
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
stopwords_ru += additional_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/w_lander/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Загрузка тестового датафрейма
#df = pd.read_csv('../pars/data_train/vkus_vils_products.csv')
df = pd.read_csv('../pars/data_test/test.csv')

In [4]:
%%time
# Предварительная обработка
df['target'] = df['stars'].apply(get_label_target) # Создаем колонку с целевой переменной "target_by_stars"
df['comment_pre_processing'] = df['comment'].apply(lambda x: clean_text(x)) # очищенными от лишних данных 
df['comment_pre_processing'] = df['comment_pre_processing'].apply(lambda x: lemmatization(x)) # применим функцию lemmatization
df['comment_pre_processing'] = df['comment_pre_processing'].apply(lambda x: ' '.join(x)) # приведем знаячения к строкам
df.dropna(subset=['comment_pre_processing'], inplace=True) # удалим пропуски в процессе лемматизации (при их наличии)

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True, max_features=None) # Применение TF-IDF к признаку 'comment_pre_processing'
df_tfidf_matrix = tfidf_vectorizer.fit_transform(df['comment_pre_processing'])
# Нормализация матрицы TF-IDF по длине документа

df_tfidf_matrix_normalized = normalize(df_tfidf_matrix, norm='l2', axis=1)
# Преобразование в разреженную матрицу
sparse_matrix = csr_matrix(df_tfidf_matrix_normalized)
# Понижение размерности TSNE
tsne = TSNE(n_components=2, random_state=42, n_jobs=-1, perplexity=20)
tsne_result = tsne.fit_transform(df_tfidf_matrix.toarray())

/tmp/ipykernel_982482/1355504995.py:23: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


CPU times: user 13.9 s, sys: 672 ms, total: 14.6 s
Wall time: 2.49 s


In [5]:
# загружаем модель
from joblib import dump, load
clf = load('models/best_model.pkl') 

In [6]:
pred = clf.predict(tsne_result)

In [7]:
df['target'].value_counts()

1    409
0    281
Name: target, dtype: int64

In [8]:
f1_score(df['target'], pred)

0.7127819548872181

In [9]:
print(classification_report(df['target'], pred))

              precision    recall  f1-score   support

           0       0.60      0.93      0.73       281
           1       0.93      0.58      0.71       409

    accuracy                           0.72       690
   macro avg       0.76      0.76      0.72       690
weighted avg       0.79      0.72      0.72       690



In [10]:
df.drop(['comment_pre_processing', 'target'], axis=1, inplace=True) # перед записью в БД удаляем 'comment_pre_processing'
df['status'] = pred # добавляем колонку с предсказаниями 'status'

In [15]:
df.sample(3)

,author,comment,date,url,product,category,stars,price,currency,weight,status
66,Сергей,Отличная зелень,18 мая 2025 · Карта: xxx51,https://vkusvill.ru/goods/salat-rukola-125-g-2...,"Салат Рукола, 125 г",Овощи,5,205,руб,/шт,0
498,Ирина,Хлеб от Пеко присутствует привкус ванили а это...,14 мая 2025 · Карта: xxx40,https://vkusvill.ru/goods/baton-nareznoy-polov...,"Батон нарезной, половинка",Особое,4,41,руб,/шт,1
421,ГАЛИЯ,Постоянно горелый,21 мая 2025 · Карта: xxx33,https://vkusvill.ru/goods/baton-nareznoy-polov...,"Батон нарезной, половинка",Особое,1,41,руб,/шт,0


In [12]:
df.to_csv('result.csv', encoding='utf8', index=False)